In [11]:
%%bash --out results
i=2
abs_path="/workspaces/reducer"

#for i in {1..20}; do
cargo build
cargo run --package reducer --bin reducer -- --query /workspaces/reducer/queries/query19/original_test.sql --test /workspaces/reducer/src/resources/native.sh

   Compiling reducer v0.1.0 (/workspaces/reducer)
 --> src/driver.rs:8:5
  |
8 | use std::sync::OnceLock;
  |     ^^^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/parser.rs:1:5
  |
1 | use log::info;
  |     ^^^^^^^^^

 --> src/reducer.rs:2:5
  |
2 | use crate::driver;
  |     ^^^^^^^^^^^^^

 --> src/reducer.rs:3:5
  |
3 | use crate::transformation;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/reducer.rs:5:5
  |
5 | use crate::utils::vec_statement_to_string;
  |     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

 --> src/reducer.rs:8:15
  |
8 | use std::io::{Error, Read};
  |               ^^^^^  ^^^^

 --> src/transformation/constant_fold.rs:4:5
  |
4 | use sqlparser::ast;
  |     ^^^^^^^^^^^^^^

 --> src/transformation/constant_fold.rs:5:38
  |
5 | use sqlparser::ast::Value::{Boolean, Null, Number};
  |                                      ^^^^

 --> src/transformation/constant_fold.rs:7:38
  |
7 |     BinaryOperator, Expr as SQLExpr, Insert, Query, SelectItem, 

CalledProcessError: Command 'b'i=2\nabs_path="/workspaces/reducer"\n\n#for i in {1..20}; do\ncargo build\ncargo run --package reducer --bin reducer -- --query /workspaces/reducer/queries/query19/original_test.sql --test /workspaces/reducer/src/resources/native.sh\n'' returned non-zero exit status 101.

In [12]:
print(results)

Current directory: /workspaces/reducer/src/resources

